In [6]:
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
import os
import cv2
import math
import numpy as np
import copy
import itertools
import csv
from collections import Counter
from collections import deque
from utils import CvFpsCalc
from matplotlib import pyplot as plt

In [7]:
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480


def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  return img

def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        #print(filename)
        os.chdir(folder)
        img = cv2.imread(filename)
        #print(img)
        if img is not None:
            img=resize_and_show(img)
            images.append(img)
    return images

In [8]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [9]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))
    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [17]:
def logging_csv(number, mode, landmark_list, point_history_list):
# if mode == 0:
#     pass
# if mode == 1 and (0 <= number <= 9):
    csv_path = 'model/keypoint_classifier/keypoint.csv'
    #csv_path = 'model/keypoint_classifier/keypoint_withoutQ&X.csv'


    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f)
        writer.writerow([number, *landmark_list])

    return

In [18]:
#Land mark collection
root_dir="C:\\Users\\HP\\Desktop\\video_to_frames\\PolishAlphabet_SignLanguage_Detection\\input"
letters=os.listdir(root_dir)
letters.sort()
label=-1
for letter in letters:
    label=label+1
    input_dir =os.path.join(root_dir,letter)
    images=load_images_from_folder(input_dir)
    os.chdir("C:\\Users\\HP\\Desktop\\Thesis\\PolishAlphabet_SignLanguage_Detection")
    for image in images:
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.7) as hands:
        
            results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))

            if results.multi_hand_landmarks is not None:
                for hand_landmarks in results.multi_hand_landmarks:
                    
                    # Landmark calculation
                    landmark_list = calc_landmark_list(image, hand_landmarks)

                    #print(landmark_list)
                    # Conversion to relative coordinates / normalized coordinates
                    pre_processed_landmark_list = pre_process_landmark(
                            landmark_list)

                    #print(pre_processed_landmark_list)
                    # Write to the dataset file
                    logging_csv(label, 1, pre_processed_landmark_list,[])
                    

In [13]:
len(letters)

26